In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('../../prices.txt', sep=r'\s+', header=None)

# Params
short_w        = 2
long_w         = 100
threshold_up   = 0.000   # 1.5%
threshold_down = 0.01    # 1.0%

# MAs
ma_short = df.rolling(window=short_w, min_periods=1).mean()
ma_long  = df.rolling(window=long_w,  min_periods=1).mean()

for inst in df.columns:
    price = df[inst]
    ms    = ma_short[inst]
    ml    = ma_long[inst]

    # normalized, unitless gap
    norm_gap = (ms - ml) / ml

    # new, scale-free conditions
    cond_green  = norm_gap >  threshold_up
    cond_red    = norm_gap < -threshold_down
    cond_yellow = ~(cond_green | cond_red)

    fig, ax = plt.subplots(figsize=(16, 5))

    # plot price & MAs
    ax.plot(price.index, price, label='Price', linewidth=1)
    ax.plot(ms.index,    ms,    label=f'MA{short_w}', linewidth=1)
    ax.plot(ml.index,    ml,    label=f'MA{long_w}', linewidth=1)

    # shading extents
    ymin, ymax = price.min(), price.max()
    ax.fill_between(price.index, ymin, ymax, where=cond_green,  color='green',  alpha=0.15)
    ax.fill_between(price.index, ymin, ymax, where=cond_yellow, color='yellow', alpha=0.15)
    ax.fill_between(price.index, ymin, ymax, where=cond_red,    color='red',    alpha=0.15)

    # labels
    ax.set_title(f'Instrument {inst}')
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Price')

    # legend above
    ax.legend(
        ['Price',
         f'MA{short_w}',
         f'MA{long_w}',
         f'Gap% > +{threshold_up*100:.1f}%',
         f'|Gap%| ≤ threshold',
         f'Gap% < -{threshold_down*100:.1f}%'],
        loc='upper center',
        bbox_to_anchor=(0.5, 1.15),
        ncol=6,
        frameon=False,
        fontsize='small'
    )

    plt.tight_layout(rect=[0, 0, 1, 0.88])
    plt.show()


In [ ]:
import sys
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ParameterGrid

# ─── 1) Make sure Python can find precision_labeller ─────────────────────────

from precision_labeller import plot_all_regimes_long

# ─── 2) Load your prices (50×750) ────────────────────────────────────────────
df = pd.read_csv("prices.txt", sep=r"\s+", header=None)
end_point = df.shape[0]
n_inst    = df.shape[1]

# ─── 3) Fixed gap‐thresholds ─────────────────────────────────────────────────
threshold_up   = 0.000   # +0.2% above
threshold_down = 0.000   # −1.5% below

# ─── 4) Define the grid of MA windows you want to try ────────────────────────
param_grid = {
    "short_w": [2, 5, 10, 20],
    "long_w":  [20, 50, 100, 200],
}

# ─── 5) Precompute the “true” regimes for each instrument ────────────────────
true_regs = {
    inst: plot_all_regimes_long(end_point, plot_graph=False, inst=inst)
    for inst in range(n_inst)
}

results = []

# ─── 6) Grid‐search loop ────────────────────────────────────────────────────
for params in ParameterGrid(param_grid):
    short_w = params["short_w"]
    long_w  = params["long_w"]

    # compute rolling MAs
    ma_s = df.rolling(window=short_w, min_periods=1).mean()
    ma_l = df.rolling(window=long_w,  min_periods=1).mean()

    scores = []
    for inst in range(n_inst):
        # raw arrays
        ms = ma_s[inst].values
        ml = ma_l[inst].values

        # unit‐less gap
        norm_gap = (ms - ml) / ml
        cond_green  = norm_gap >  threshold_up
        cond_red    = norm_gap < -threshold_down

        # predicted regimes: 2=bull, 1=neutral, 0=bear
        pred = np.where(cond_green, 2,
                        np.where(cond_red,   0, 1))

        true = true_regs[inst]
        pred = pred[: len(true)]      # align lengths

        scores.append(accuracy_score(true, pred))

    results.append({
        "short_w": short_w,
        "long_w":  long_w,
        "accuracy": np.mean(scores)
    })

# ─── 7) Summarize results ────────────────────────────────────────────────────
results_df = pd.DataFrame(results)
best = results_df.loc[results_df["accuracy"].idxmax()]

print("=== MA Grid Search Results ===")
print(results_df.sort_values("accuracy", ascending=False).to_string(index=False))
print(f"\n🏅  Best MA windows: short_w={best.short_w}, long_w={best.long_w} "
      f"(avg accuracy={best.accuracy:.4f})")


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load data
df = pd.read_csv('../../prices.txt', sep=r'\s+', header=None)

# Params
short_w        = 15
long_w         = 30
threshold_up   = 0.000   # 0.0% above
threshold_down = 0.00    # 1.0% below

# EMAs
ema_short = df.ewm(span=short_w, adjust=False).mean()
ema_long  = df.ewm(span=long_w,  adjust=False).mean()

for inst in df.columns:
    price = df[inst]
    es    = ema_short[inst]
    el    = ema_long[inst]

    # normalized, unitless gap
    norm_gap = (es - el) / el

    # scale-free regimes
    cond_green  = norm_gap >  threshold_up
    cond_red    = norm_gap < -threshold_down
    cond_yellow = ~(cond_green | cond_red)

    fig, ax = plt.subplots(figsize=(16, 5))

    # plot price & EMAs
    ax.plot(price.index, price, label='Price', linewidth=1)
    ax.plot(es.index,    es,    label=f'EMA{short_w}', linewidth=1)
    ax.plot(el.index,    el,    label=f'EMA{long_w}', linewidth=1)

    # shading
    ymin, ymax = price.min(), price.max()
    ax.fill_between(price.index, ymin, ymax, where=cond_green,  color='green',  alpha=0.15)
    ax.fill_between(price.index, ymin, ymax, where=cond_yellow, color='yellow', alpha=0.15)
    ax.fill_between(price.index, ymin, ymax, where=cond_red,    color='red',    alpha=0.15)

    # labels
    ax.set_title(f'Instrument {inst}')
    ax.set_xlabel('Timestep')
    ax.set_ylabel('Price')

    # legend centered above
    ax.legend(
        ['Price',
         f'EMA{short_w}',
         f'EMA{long_w}',
         f'Gap% > +{threshold_up*100:.1f}%',
         f'|Gap%| ≤ threshold',
         f'Gap% < -{threshold_down*100:.1f}%'],
        loc='upper center',
        bbox_to_anchor=(0.5, 1.15),
        ncol=6,
        frameon=False,
        fontsize='small'
    )

    plt.tight_layout(rect=[0, 0, 1, 0.88])
    plt.show()


In [ ]:
import pandas as pd
import mplfinance as mpf
from pathlib import Path

# ─── Load Close-Only Data ──────────────────────────────────────────────────────
price_path = Path("../../prices.txt")
df_close = pd.read_csv(price_path, sep=r"\s+", header=None)

# ─── Prepare OHLC from Close-only ─────────────────────────────────────────────
# Example for instrument 0
close = df_close[0]
# Create a DateTime index (replace with your real timestamps if available)
dates = pd.date_range(end=pd.Timestamp.today(), periods=len(close), freq='D')

ohlc = pd.DataFrame(index=dates)
ohlc['Close'] = close.values
# Synthetic Open = previous Close (first Open = first Close)
ohlc['Open']  = ohlc['Close'].shift(1).fillna(ohlc['Close'].iloc[0])
# High/Low = max/min of Open/Close
ohlc['High']  = ohlc[['Open', 'Close']].max(axis=1)
ohlc['Low']   = ohlc[['Open', 'Close']].min(axis=1)

# ─── Plot Candlestick Chart with Wicks ────────────────────────────────────────
N = 50  # however many candles you want

# … after you’ve built your full `ohlc` DataFrame …
ohlc_recent = ohlc.tail(N)

mpf.plot(
    ohlc_recent,
    type='candle',
    style='charles',
    title=f'Instrument 0 – Last {N} Candles',
    ylabel='Price',
    volume=False,
    figsize=(12, 6)
)


# To loop over all instruments, wrap the above in a for-loop over df_close.columns,
# generating one ohlc DataFrame per instrument and plotting similarly.


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 7       # pick instrument 0 … 49
LAST_N        = 120     # number of last points to plot
FIGSIZE       = (12, 6)
COLOR_PRICE   = 'tab:blue'
COLOR_HA      = 'tab:orange'
COLOR_EMA10   = 'tab:green'
COLOR_EMA30   = 'tab:red'

# ─── 1) Load only the selected instrument’s close prices ────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        if len(vals) <= INSTRUMENT_ID:
            val = vals[-1]
        else:
            val = vals[INSTRUMENT_ID]
        closes.append(val)

T = len(closes)
dates = list(range(T))  # or replace with real datetime list

# ─── 2) Build synthetic OHLC ─────────────────────────────────────────────────
opens = [0.0]*T
highs = [0.0]*T
lows  = [0.0]*T

for t in range(T):
    c = closes[t]
    o = closes[t-1] if t>0 else c
    opens[t] = o
    highs[t] = max(o, c)
    lows[t]  = min(o, c)

# ─── 3) Compute Heiken‐Ashi ────────────────────────────────────────────────
ha_open  = [0.0]*T
ha_close = [0.0]*T

for t in range(T):
    o, h, l, c = opens[t], highs[t], lows[t], closes[t]
    hac = (o + h + l + c) / 4.0
    if t == 0:
        hao = (o + c) / 2.0
    else:
        hao = (ha_open[t-1] + ha_close[t-1]) / 2.0
    ha_open[t]  = hao
    ha_close[t] = hac

# ─── 4) Compute 10‐ and 30‐period EMAs ─────────────────────────────────────
def compute_ema(series, period):
    ema = [series[0]]
    alpha = 2 / (period + 1)
    for price in series[1:]:
        ema.append(alpha * price + (1 - alpha) * ema[-1])
    return ema



# ─── 5) Slice to last LAST_N points ────────────────────────────────────────
start = max(0, T - LAST_N)
plot_dates    = dates[start:]
plot_closes   = closes[start:]
plot_ha_close = ha_close[start:]



# ─── 6) Plot all on the same axis ─────────────────────────────────────────
plt.figure(figsize=FIGSIZE)
plt.plot(plot_dates, plot_closes,   label="Raw Close", color=COLOR_PRICE, linewidth=1)
plt.plot(plot_dates, plot_ha_close, label="HA Close",  color=COLOR_HA,    linewidth=1)


plt.title(f"Instrument {INSTRUMENT_ID}: Last {len(plot_dates)} Points")
plt.xlabel("Time (index)")
plt.ylabel("Price")
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE     = "prices.txt"
INSTRUMENT_ID  = 7        # pick instrument 0 … 49
LAST_N         = 200      # number of last points to plot
SMOOTH_PERIOD  = 5       # EMA length for smoothing HA
FIGSIZE        = (12, 6)
COLOR_CLOSE    = 'tab:blue'
COLOR_HA       = 'tab:orange'
COLOR_SMHA     = 'tab:purple'

# ─── 1) Load closes ─────────────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

T = len(closes)
dates = list(range(T))

# ─── 2) Build synthetic OHLC ───────────────────────────────────────────────
opens = [0.0]*T
highs = [0.0]*T
lows  = [0.0]*T
for t in range(T):
    o = closes[t-1] if t>0 else closes[0]
    c = closes[t]
    opens[t] = o
    highs[t] = max(o, c)
    lows[t]  = min(o, c)

# ─── 3) Compute standard Heiken-Ashi ──────────────────────────────────────
ha_open  = [0.0]*T
ha_close = [0.0]*T
for t in range(T):
    o, h, l, c = opens[t], highs[t], lows[t], closes[t]
    # HA close
    ha_c = (o + h + l + c) / 4.0
    # HA open
    ha_o = (ha_open[t-1] + ha_close[t-1]) / 2.0 if t>0 else (o + c)/2.0
    ha_open[t]  = ha_o
    ha_close[t] = ha_c

# ─── 4) EMA helper ─────────────────────────────────────────────────────────
def compute_ema(series, period):
    alpha = 2.0 / (period + 1)
    ema = [series[0]]
    for v in series[1:]:
        ema.append(alpha * v + (1-alpha) * ema[-1])
    return ema

# ─── 5) Compute Smoothed HA ────────────────────────────────────────────────
smha_open  = compute_ema(ha_open,  SMOOTH_PERIOD)
smha_close = compute_ema(ha_close, SMOOTH_PERIOD)

# ─── 6) Slice to last LAST_N points ────────────────────────────────────────
start        = max(0, T - LAST_N)
xs           = dates[start:]
raw_close_s  = closes[start:]
ha_close_s   = ha_close[start:]
smha_close_s = smha_close[start:]

# ─── 7) Plot ───────────────────────────────────────────────────────────────
plt.figure(figsize=FIGSIZE)
plt.plot(xs, raw_close_s,  label="Raw Close",          color=COLOR_CLOSE, linewidth=1)
plt.plot(xs, ha_close_s,   label="HA Close",           color=COLOR_HA,    linewidth=1)
plt.plot(xs, smha_close_s, label=f"Smoothed HA ({SMOOTH_PERIOD})", color=COLOR_SMHA, linewidth=1.5)

plt.title(f"Instrument {INSTRUMENT_ID}: Last {len(xs)} Points")
plt.xlabel("Time (index)")
plt.ylabel("Price")
plt.legend(loc="lower left")
plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 12     # pick instrument 0 … 49
LAST_N        = 120     # number of last points to plot
ATR_PERIOD    = 50      # look-back for ATR
MULTIPLIER    = 1       # SuperTrend multiplier
FIGSIZE       = (12, 6)
COLOR_PRICE   = 'tab:blue'
COLOR_UP      = 'tab:green'
COLOR_DOWN    = 'tab:red'

# ─── 1) Load the selected instrument’s close prices ────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

T = len(closes)
dates = list(range(T))

# ─── 2) Build synthetic OHLC ───────────────────────────────────────────────
opens = [0.0]*T
highs = [0.0]*T
lows  = [0.0]*T

for t in range(T):
    c = closes[t]
    o = closes[t-1] if t>0 else c
    opens[t] = o
    highs[t] = max(o, c)
    lows[t]  = min(o, c)

# ─── 3) Compute True Range (TR) ────────────────────────────────────────────
trs = [highs[0] - lows[0]]
for t in range(1, T):
    tr1 = highs[t] - lows[t]
    tr2 = abs(highs[t] - closes[t-1])
    tr3 = abs(lows[t]  - closes[t-1])
    trs.append(max(tr1, tr2, tr3))

# ─── 4) Compute ATR using Wilder’s smoothing ───────────────────────────────
atrs = []
first_atr = sum(trs[:ATR_PERIOD]) / min(ATR_PERIOD, len(trs))
atrs.append(first_atr)
for t in range(1, T):
    prev = atrs[-1]
    atrs.append((prev * (ATR_PERIOD - 1) + trs[t]) / ATR_PERIOD)

# ─── 5) Compute SuperTrend ────────────────────────────────────────────────
basic_ub = [ (highs[t]+lows[t]) / 2 + MULTIPLIER * atrs[t] for t in range(T) ]
basic_lb = [ (highs[t]+lows[t]) / 2 - MULTIPLIER * atrs[t] for t in range(T) ]

final_ub = [basic_ub[0]]
final_lb = [basic_lb[0]]
supertrend = [basic_ub[0]]
direction = [-1]  # -1 = downtrend, +1 = uptrend

for t in range(1, T):
    # carry-forward Logic
    if basic_ub[t] < final_ub[t-1] or closes[t-1] > final_ub[t-1]:
        final_ub.append(basic_ub[t])
    else:
        final_ub.append(final_ub[t-1])

    if basic_lb[t] > final_lb[t-1] or closes[t-1] < final_lb[t-1]:
        final_lb.append(basic_lb[t])
    else:
        final_lb.append(final_lb[t-1])

    # pick which band to use and record direction
    if closes[t] <= final_ub[t]:
        supertrend.append(final_ub[t])
        direction.append(-1)
    else:
        supertrend.append(final_lb[t])
        direction.append(+1)

# ─── 6) Slice to last LAST_N points ────────────────────────────────────────
start = max(0, T - LAST_N)
xs    = dates[start:]
ys    = closes[start:]
sts   = supertrend[start:]
dirs  = direction[start:]

# ─── 7) Plot price & SuperTrend with Up=Green, Down=Red ──────────────────
plt.figure(figsize=FIGSIZE)
plt.plot(xs, ys, label="Close Price", color=COLOR_PRICE, linewidth=1)

for i in range(len(xs)-1):
    col = COLOR_UP if dirs[i] > 0 else COLOR_DOWN
    plt.plot(xs[i:i+2], sts[i:i+2], color=col, linewidth=1.5)

plt.title(f"Instrument {INSTRUMENT_ID}: Close & SuperTrend (Last {len(xs)} Points)")
plt.xlabel("Time (index)")
plt.ylabel("Price")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 12      # pick instrument 0 … 49

# Index‐range to plot (inclusive start, exclusive end):
START_IDX     = 500     
END_IDX       = 750     

# Define three (ATR_PERIOD, MULTIPLIER, label) tuples:
ST_PARAMS = [
    (100, 3,  "ST (10,3)"),
    (90, 2,  "ST (20,2)"),
    (80, 1,  "ST (50,1)"),
]

FIGSIZE       = (14, 6)
COLOR_PRICE   = 'tab:blue'
COLOR_UP      = 'tab:green'
COLOR_DOWN    = 'tab:red'

# ─── 1) Load closes for the chosen instrument ──────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

T = len(closes)
dates = list(range(T))

# ─── 2) Build synthetic OHLC ───────────────────────────────────────────────
opens = [0.0]*T
highs = [0.0]*T
lows  = [0.0]*T
for t in range(T):
    c = closes[t]
    o = closes[t-1] if t>0 else c
    opens[t], highs[t], lows[t] = o, max(o,c), min(o,c)

# ─── 3) Precompute True Range once ────────────────────────────────────────
trs = [highs[0] - lows[0]]
for t in range(1, T):
    tr1 = highs[t] - lows[t]
    tr2 = abs(highs[t] - closes[t-1])
    tr3 = abs(lows[t]  - closes[t-1])
    trs.append(max(tr1, tr2, tr3))

def compute_supertrend(trs, highs, lows, closes, atr_period, mult):
    """Returns (supertrend, direction) for the given params."""
    # 4a) ATR via Wilder’s smoothing
    atrs = []
    first = sum(trs[:atr_period]) / min(atr_period, len(trs))
    atrs.append(first)
    for i in range(1, len(trs)):
        atrs.append((atrs[-1] * (atr_period - 1) + trs[i]) / atr_period)

    # 4b) Basic bands
    basic_ub = [ (highs[i]+lows[i])/2 + mult * atrs[i] for i in range(len(trs)) ]
    basic_lb = [ (highs[i]+lows[i])/2 - mult * atrs[i] for i in range(len(trs)) ]

    # 4c) Final bands + SuperTrend logic
    final_ub = [basic_ub[0]]
    final_lb = [basic_lb[0]]
    st       = [basic_ub[0]]
    dirn     = [-1]  # -1 = down, +1 = up

    for i in range(1, len(trs)):
        # carry‐forward
        if basic_ub[i] < final_ub[i-1] or closes[i-1] > final_ub[i-1]:
            final_ub.append(basic_ub[i])
        else:
            final_ub.append(final_ub[i-1])

        if basic_lb[i] > final_lb[i-1] or closes[i-1] < final_lb[i-1]:
            final_lb.append(basic_lb[i])
        else:
            final_lb.append(final_lb[i-1])

        # pick band & direction
        if closes[i] <= final_ub[i]:
            st.append(final_ub[i]); dirn.append(-1)
        else:
            st.append(final_lb[i]); dirn.append(+1)

    return st, dirn

# ─── 5) Slice to the requested index‐window ────────────────────────────────
xs  = dates[START_IDX:END_IDX]
ys  = closes[START_IDX:END_IDX]

# ─── 6) Plot everything ────────────────────────────────────────────────────
plt.figure(figsize=FIGSIZE)
plt.plot(xs, ys, label="Close Price", color=COLOR_PRICE, linewidth=1.5)

# For each ST variant, compute & plot
for atr_p, mult, label in ST_PARAMS:
    st, dirn = compute_supertrend(trs, highs, lows, closes, atr_p, mult)
    sts = st[START_IDX:END_IDX]
    dns = dirn[START_IDX:END_IDX]

    # draw it piecewise so it shifts color by dirn
    for i in range(len(xs)-1):
        c = COLOR_UP if dns[i] > 0 else COLOR_DOWN
        plt.plot(xs[i:i+2], sts[i:i+2], color=c, linewidth=1, alpha=0.8)

    # add a dummy for the legend
    plt.plot([], [], color='black', linewidth=1, label=label)

plt.title(f"Instr {INSTRUMENT_ID}: Close + 3×SuperTrend [{START_IDX}:{END_IDX}]")
plt.xlabel("Index")
plt.ylabel("Price")
plt.legend(loc="upper left")
plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 33      # pick instrument 0 … 49

# Index‐range to plot (inclusive start, exclusive end):
START_IDX     = 500     
END_IDX       = 750     

# SuperTrend variants: (ATR_PERIOD, MULTIPLIER, label)
ST_PARAMS = [
    (50, 3,  "ST (50,3)"),
    (40, 2,  "ST (40,2)"),
    (30, 1,  "ST (30,1)"),
]

FIGSIZE       = (14, 8)
COLOR_PRICE   = 'tab:blue'
COLOR_UP      = 'tab:green'
COLOR_DOWN    = 'tab:red'
ALPHA_BG      = 0.15

# ─── 1) Load closes ─────────────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])
T = len(closes)
dates = list(range(T))

# ─── 2) Synthetic OHLC ─────────────────────────────────────────────────────
opens = [closes[i-1] if i>0 else closes[0] for i in range(T)]
highs = [max(o,c) for o,c in zip(opens, closes)]
lows  = [min(o,c) for o,c in zip(opens, closes)]

# ─── 3) True Range ─────────────────────────────────────────────────────────
trs = [highs[0] - lows[0]]
for i in range(1, T):
    tr1 = highs[i] - lows[i]
    tr2 = abs(highs[i] - closes[i-1])
    tr3 = abs(lows[i]  - closes[i-1])
    trs.append(max(tr1, tr2, tr3))

# ─── 4) EMA helper ─────────────────────────────────────────────────────────
def compute_ema(series, period):
    α = 2.0 / (period + 1)
    ema = [series[0]]
    for price in series[1:]:
        ema.append(α * price + (1 - α) * ema[-1])
    return ema

# ─── 5) Compute EMA12 & EMA26 ──────────────────────────────────────────────
ema12 = compute_ema(closes, 2)
ema26 = compute_ema(closes, 40)

# ─── 6) Robust SuperTrend ──────────────────────────────────────────────────
def compute_supertrend(trs, highs, lows, closes, atr_period, mult):
    n = len(trs)
    atrs = [None]*n
    if n > atr_period:
        initial = sum(trs[1:atr_period+1]) / atr_period
        atrs[atr_period] = initial
        for i in range(atr_period+1, n):
            atrs[i] = (atrs[i-1]*(atr_period-1) + trs[i]) / atr_period

    basic_ub = [None]*n
    basic_lb = [None]*n
    for i in range(n):
        if atrs[i] is not None:
            mid = (highs[i] + lows[i]) / 2
            basic_ub[i] = mid + mult * atrs[i]
            basic_lb[i] = mid - mult * atrs[i]

    final_ub = [None]*n
    final_lb = [None]*n
    st       = [None]*n
    dirn     = [0]*n

    # first valid bar
    start = next((i for i,v in enumerate(atrs) if v is not None), n-1)
    final_ub[start] = basic_ub[start]
    final_lb[start] = basic_lb[start]
    st[start]       = basic_ub[start]
    dirn[start]     = -1

    # backward fill
    for i in range(start-1, -1, -1):
        final_ub[i] = final_ub[start]
        final_lb[i] = final_lb[start]
        st[i]       = st[start]
        dirn[i]     = dirn[start]

    # forward iterate
    for i in range(start+1, n):
        if basic_ub[i] < final_ub[i-1] or closes[i-1] > final_ub[i-1]:
            final_ub[i] = basic_ub[i]
        else:
            final_ub[i] = final_ub[i-1]

        if basic_lb[i] > final_lb[i-1] or closes[i-1] < final_lb[i-1]:
            final_lb[i] = basic_lb[i]
        else:
            final_lb[i] = final_lb[i-1]

        if closes[i] <= final_ub[i]:
            st[i]   = final_ub[i]
            dirn[i] = -1
        else:
            st[i]   = final_lb[i]
            dirn[i] = +1

    return st, dirn

# ─── 7) Compute all 3 SuperTrends ─────────────────────────────────────────
all_st = []
all_dir = []
for atr_p, mult, label in ST_PARAMS:
    st, dirn = compute_supertrend(trs, highs, lows, closes, atr_p, mult)
    all_st.append((st, label))
    all_dir.append(dirn)

# ─── 8) Consensus: +1 if all up, -1 if all down, else 0 ──────────────────
consensus = []
for i in range(T):
    if all(d[i] > 0 for d in all_dir):
        consensus.append(+1)
    elif all(d[i] < 0 for d in all_dir):
        consensus.append(-1)
    else:
        consensus.append(0)

# ─── 9) Slice to window ───────────────────────────────────────────────────
xs    = dates[START_IDX:END_IDX]
ys    = closes[START_IDX:END_IDX]
ema12_s = ema12[START_IDX:END_IDX]
ema26_s = ema26[START_IDX:END_IDX]
sts_s = [(st[START_IDX:END_IDX], dirn[START_IDX:END_IDX], label)
         for (st,label), dirn in zip(all_st, all_dir)]
cons_s = consensus[START_IDX:END_IDX]

# ─── 10) Plot ──────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=FIGSIZE, sharex=True)

# Top: Close + SuperTrend
ax1.plot(xs, ys, color=COLOR_PRICE, label="Close", linewidth=1.5)
for st_slice, dir_slice, label in sts_s:
    for i in range(len(xs)-1):
        c = COLOR_UP if dir_slice[i] > 0 else COLOR_DOWN
        ax1.plot(xs[i:i+2], st_slice[i:i+2], color=c, linewidth=1)
    ax1.plot([], [], color='black', label=label)
ax1.set_title(f"Instr {INSTRUMENT_ID}: Close + 3×SuperTrend [{START_IDX}:{END_IDX}]")
ax1.set_ylabel("Price")
ax1.legend(loc="upper left")
ax1.grid(alpha=0.3)

# Bottom: Close + filtered consensus shading
ax2.plot(xs, ys, color=COLOR_PRICE, label="Close", linewidth=1.5)
for i in range(len(xs)-1):
    # only long if EMA12 > EMA26 AND consensus long
    if cons_s[i] == +1 and ema12_s[i] > ema26_s[i]:
        ax2.axvspan(xs[i], xs[i+1], facecolor=COLOR_UP, alpha=ALPHA_BG)
    # only short if EMA12 < EMA26 AND consensus short
    elif cons_s[i] == -1 and ema12_s[i] < ema26_s[i]:
        ax2.axvspan(xs[i], xs[i+1], facecolor=COLOR_DOWN, alpha=ALPHA_BG)

ax2.set_title("Filtered Consensus Zones (EMA12/EMA26)")
ax2.set_xlabel("Index")
ax2.set_ylabel("Price")
ax2.legend(loc="upper left")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── CONFIG ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 12        # choose 0…49
FIGSIZE       = (14, 8)

# MA & MACD settings
MA200_PERIOD  = 200
EMA_FAST      = 12
EMA_SLOW      = 26
EMA_SIGNAL    = 9

# Colors
COLOR_PRICE   = 'tab:blue'
COLOR_MA200    = 'tab:orange'
COLOR_MACD     = 'tab:green'
COLOR_SIGNAL   = 'tab:red'
COLOR_HIST_UP  = 'tab:green'
COLOR_HIST_DN  = 'tab:red'
ALPHA_HIST    = 0.5

# ─── 1) LOAD CLOSE PRICES ───────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

dates = list(range(len(closes)))

# ─── 2) SIMPLE 200-DAY MOVING AVERAGE ──────────────────────────────────────
ma200 = []
for i in range(len(closes)):
    if i < MA200_PERIOD - 1:
        ma200.append(float('nan'))
    else:
        window = closes[i-MA200_PERIOD+1 : i+1]
        ma200.append(sum(window) / MA200_PERIOD)

# ─── 3) EMA FUNCTION ───────────────────────────────────────────────────────
def compute_ema(series, period):
    """Wilder EMA: α=2/(period+1)"""
    ema = [series[0]]
    α   = 2.0 / (period + 1)
    for price in series[1:]:
        ema.append(α * price + (1 - α) * ema[-1])
    return ema

# ─── 4) COMPUTE MACD LINE, SIGNAL LINE, HISTOGRAM ─────────────────────────
ema_fast   = compute_ema(closes, EMA_FAST)
ema_slow   = compute_ema(closes, EMA_SLOW)
macd_line  = [f - s for f, s in zip(ema_fast, ema_slow)]
signal_line= compute_ema(macd_line, EMA_SIGNAL)
hist       = [m - s for m, s in zip(macd_line, signal_line)]

# ─── 5) PLOT ───────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=FIGSIZE, sharex=True)

# Top: Price + MA200
ax1.plot(dates, closes, label="Close Price", color=COLOR_PRICE)
ax1.plot(dates, ma200,  label="200-day MA", color=COLOR_MA200)
ax1.set_title(f"Instrument {INSTRUMENT_ID} — Price & 200-Day MA")
ax1.set_ylabel("Price")
ax1.legend(loc="upper left")
ax1.grid(True, alpha=0.3)

# Bottom: MACD, Signal, Histogram
ax2.plot(dates, macd_line,  label="MACD (12–26 EMA)", color=COLOR_MACD)
ax2.plot(dates, signal_line,label="Signal (9 EMA of MACD)", color=COLOR_SIGNAL)

# Fill histogram positive / negative
ax2.fill_between(
    dates,
    hist,
    0,
    where=[h >= 0 for h in hist],
    facecolor=COLOR_HIST_UP,
    alpha=ALPHA_HIST,
    interpolate=True,
)
ax2.fill_between(
    dates,
    hist,
    0,
    where=[h < 0 for h in hist],
    facecolor=COLOR_HIST_DN,
    alpha=ALPHA_HIST,
    interpolate=True,
)

ax2.set_title("MACD Indicator")
ax2.set_xlabel("Time (index)")
ax2.set_ylabel("MACD Value")
ax2.legend(loc="upper left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── CONFIG ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 11      # pick instrument 0…49
LAST_N        = 300      # number of last bars to plot
EMA200_PERIOD = 200
EMA_FAST      = 20
EMA_SLOW      = 50
EMA_SIGNAL    = 9

FIGSIZE       = (14, 8)
COLOR_PRICE   = 'tab:blue'
COLOR_EMA200   = 'tab:orange'
COLOR_MACD     = 'tab:green'
COLOR_SIGNAL   = 'tab:red'
COLOR_HIST_UP  = 'tab:green'
COLOR_HIST_DN  = 'tab:red'
ALPHA_HIST    = 0.5
ALPHA_BG      = 0.15

# ─── 1) LOAD CLOSE PRICES ─────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

dates = list(range(len(closes)))

# ─── 2) EMA FUNCTION ─────────────────────────────────────────────────────
def compute_ema(series, period):
    α   = 2.0 / (period + 1)
    ema = [series[0]]
    for price in series[1:]:
        ema.append(α * price + (1 - α) * ema[-1])
    return ema

# ─── 3) COMPUTE EMA200, MACD & SIGNAL ────────────────────────────────────
ema200     = compute_ema(closes, EMA200_PERIOD)
ema_fast   = compute_ema(closes, EMA_FAST)
ema_slow   = compute_ema(closes, EMA_SLOW)
macd_line  = [f - s for f, s in zip(ema_fast, ema_slow)]
signal_line= compute_ema(macd_line, EMA_SIGNAL)
hist       = [m - s for m, s in zip(macd_line, signal_line)]

# ─── 4) GENERATE POSITION SIGNALS ────────────────────────────────────────
#  0 = flat, +1 = long, -1 = short
pos = [0]
for i in range(1, len(macd_line)):
    prev, cur = macd_line[i-1], macd_line[i]
    prev_sig, cur_sig = signal_line[i-1], signal_line[i]

    # detect crosses
    cross_up   = prev <= prev_sig and cur > cur_sig
    cross_down = prev >= prev_sig and cur < cur_sig

    if pos[-1] == 0:
        # entry rules
        if cross_up   and cur < 0:
            pos.append( +1 )   # go long
        elif cross_down and cur > 0:
            pos.append( -1 )   # go short
        else:
            pos.append(0)
    elif pos[-1] == +1:
        # exit long on a cross down
        pos.append(0 if cross_down else +1)
    else:  # pos[-1] == -1
        # exit short on a cross up
        pos.append(0 if cross_up else -1)

# ─── 5) SLICE TO LAST_N ───────────────────────────────────────────────────
start = max(0, len(dates) - LAST_N)
xs    = dates[start:]
ys    = closes[start:]
ema2  = ema200[start:]
macd  = macd_line[start:]
sig   = signal_line[start:]
h    = hist[start:]
p    = pos[start:]

# ─── 6) PLOT ───────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=FIGSIZE, sharex=True)

# Top: Price + EMA200 + buy/sell shading
for i in range(len(xs)-1):
    color = 'tab:green' if p[i] > 0 else ('tab:red' if p[i] < 0 else None)
    if color:
        ax1.axvspan(xs[i], xs[i+1], facecolor=color, alpha=ALPHA_BG)

ax1.plot(xs, ys, label="Close Price", color=COLOR_PRICE, linewidth=1.5)
ax1.plot(xs, ema2, label="EMA 200",    color=COLOR_EMA200, linewidth=1.5)
ax1.set_title(f"Instr {INSTRUMENT_ID}: Price + EMA200 with Buy/Sell Zones")
ax1.set_ylabel("Price")
ax1.legend(loc="upper left")
ax1.grid(True, alpha=0.3)

# Bottom: MACD, Signal & histogram
ax2.plot(xs, macd, label="MACD (12–26 EMA)", color=COLOR_MACD)
ax2.plot(xs, sig,  label="Signal (9 EMA)",   color=COLOR_SIGNAL)
ax2.fill_between(xs, h, 0, where=[x>=0 for x in h],
                 facecolor=COLOR_HIST_UP, alpha=ALPHA_HIST, interpolate=True)
ax2.fill_between(xs, h, 0, where=[x< 0 for x in h],
                 facecolor=COLOR_HIST_DN, alpha=ALPHA_HIST, interpolate=True)
ax2.set_title("MACD Indicator")
ax2.set_xlabel("Time (index)")
ax2.set_ylabel("MACD Value")
ax2.legend(loc="upper left")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── CONFIG ────────────────────────────────────────────────────────────────
PRICE_FILE     = "prices.txt"
INSTRUMENT_ID  = 1       # pick instrument 0…49
START_IDX      = 500     # inclusive start of the range to plot
END_IDX        = 750      # exclusive end of the range to plot
ADX_PERIOD     = 14       # look-back for ADX (standard: 14 bars)

FIGSIZE        = (14, 8)
COLOR_PRICE    = 'tab:blue'
COLOR_ADX      = 'tab:purple'
COLOR_DI_PLUS  = 'tab:green'
COLOR_DI_MINUS = 'tab:red'
COLOR_THRESH   = 'gray'

# ─── 1) LOAD CLOSE PRICES ───────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        if len(vals) > INSTRUMENT_ID:
            closes.append(vals[INSTRUMENT_ID])
        else:
            closes.append(vals[-1])
dates = list(range(len(closes)))

# ─── 2) SYNTHETIC OHLC FROM CLOSE ONLY ──────────────────────────────────────
T = len(closes)
opens = [closes[i-1] if i>0 else closes[0] for i in range(T)]
highs = [max(op, cl) for op, cl in zip(opens, closes)]
lows  = [min(op, cl) for op, cl in zip(opens, closes)]

# ─── 3) TRUE RANGE (TR), +DM, -DM ──────────────────────────────────────────
tr  = [highs[0] - lows[0]]
pdm = [0.0]
ndm = [0.0]
for i in range(1, T):
    tr.append(max(
        highs[i] - lows[i],
        abs(highs[i] - closes[i-1]),
        abs(lows[i]  - closes[i-1])
    ))
    up_move   = highs[i] - highs[i-1]
    down_move = lows[i-1]  - lows[i]
    pdm.append(up_move   if up_move   > down_move and up_move   > 0 else 0.0)
    ndm.append(down_move if down_move > up_move   and down_move > 0 else 0.0)

# ─── 4) WILDER SMOOTHED ATR, +DM_S, -DM_S ─────────────────────────────────
atr = []
pdm_s = []
ndm_s = []
# first values: simple sum over first ADX_PERIOD bars
atr.append(sum(tr[1:ADX_PERIOD+1]) / ADX_PERIOD)
pdm_s.append(sum(pdm[1:ADX_PERIOD+1]) / ADX_PERIOD)
ndm_s.append(sum(ndm[1:ADX_PERIOD+1]) / ADX_PERIOD)

for i in range(ADX_PERIOD+1, T):
    atr.append((atr[-1] * (ADX_PERIOD-1) + tr[i])    / ADX_PERIOD)
    pdm_s.append((pdm_s[-1] * (ADX_PERIOD-1) + pdm[i])/ ADX_PERIOD)
    ndm_s.append((ndm_s[-1] * (ADX_PERIOD-1) + ndm[i])/ ADX_PERIOD)

# pad the beginning so indices align
atr   = [None]*(ADX_PERIOD+1) + atr
pdm_s = [None]*(ADX_PERIOD+1) + pdm_s
ndm_s = [None]*(ADX_PERIOD+1) + ndm_s

# ─── 5) +DI, -DI, DX ───────────────────────────────────────────────────────
plus_di  = [100 * pdm_s[i] / atr[i] if atr[i] and i< len(atr) else None
            for i in range(T)]
minus_di = [100 * ndm_s[i] / atr[i] if atr[i] and i< len(atr) else None
            for i in range(T)]
dx = [
    100 * abs(plus_di[i] - minus_di[i]) / (plus_di[i] + minus_di[i])
    if plus_di[i] is not None and minus_di[i] is not None and (plus_di[i] + minus_di[i]) != 0
    else None
    for i in range(T)
]

# ─── 6) ADX (Wilder’s smoothing of DX) ────────────────────────────────────
adx = [None]*(ADX_PERIOD*2+1)
# first ADX is average of first ADX_PERIOD DX values starting at bar 2*period
first_adx = sum(dx[ADX_PERIOD+1:ADX_PERIOD*2+1]) / ADX_PERIOD
adx.append(first_adx)
for i in range(ADX_PERIOD*2+2, T):
    adx.append((adx[-1] * (ADX_PERIOD-1) + dx[i]) / ADX_PERIOD)
# pad to length T
adx = adx + [None]*(T - len(adx))

# ─── 7) SLICE TO YOUR SPECIFIED RANGE ─────────────────────────────────────
xs      = dates[START_IDX:END_IDX]
price_s = closes[START_IDX:END_IDX]
adx_s   = adx[START_IDX:END_IDX]
plus_s  = plus_di[START_IDX:END_IDX]
minus_s = minus_di[START_IDX:END_IDX]

# ─── 8) PLOT PRICE + ADX PANEL ────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=FIGSIZE, sharex=True)

# Top panel: price
ax1.plot(xs, price_s, label="Close Price", color=COLOR_PRICE)
ax1.set_title(f"Instrument {INSTRUMENT_ID}: Price and ADX [{START_IDX}:{END_IDX}]")
ax1.set_ylabel("Price")
ax1.legend(loc="upper left")
ax1.grid(alpha=0.3)

# Bottom panel: ADX and DI lines
ax2.plot(xs, adx_s,   label="ADX",      color=COLOR_ADX,    linewidth=1.5)
ax2.plot(xs, plus_s,  label="+DI",      color=COLOR_DI_PLUS, linewidth=1)
ax2.plot(xs, minus_s, label="-DI",      color=COLOR_DI_MINUS,linewidth=1)
ax2.axhline(25, color=COLOR_THRESH, linestyle='--', label="Threshold (25)")
ax2.set_ylabel("Value")
ax2.set_xlabel("Time (index)")
ax2.legend(loc="upper left")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE     = "prices.txt"
INSTRUMENT_ID  = 12      # pick instrument 0…49

# Index‐range to plot (inclusive start, exclusive end):
START_IDX      = 500
END_IDX        = 750

# Donchian channel period:
DC_PERIOD      = 30

FIGSIZE        = (14, 8)
COLOR_PRICE    = 'tab:blue'
COLOR_DC_UPPER = 'tab:orange'
COLOR_DC_LOWER = 'tab:orange'
COLOR_LONG_BG  = 'tab:green'
COLOR_SHORT_BG = 'tab:red'
ALPHA_BG       = 0.15

# ─── 1) Load close prices ───────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals) > INSTRUMENT_ID else vals[-1])

T = len(closes)
dates = list(range(T))

# ─── 2) Build synthetic OHLC ───────────────────────────────────────────────
opens = [closes[i-1] if i>0 else closes[0] for i in range(T)]
highs = [max(o,c) for o,c in zip(opens, closes)]
lows  = [min(o,c) for o,c in zip(opens, closes)]

# ─── 3) Compute Donchian Channels ──────────────────────────────────────────
upper = [None]*T
lower = [None]*T
for i in range(DC_PERIOD - 1, T):
    upper[i] = max(highs[i-DC_PERIOD+1 : i+1])
    lower[i] = min(lows[i-DC_PERIOD+1  : i+1])

# ─── 4) Generate breakout positions ────────────────────────────────────────
#  0 = flat, +1 = long, -1 = short
pos = [0]*T
for i in range(1, T):
    if pos[i-1] == 0:
        # enter long on close above upper band
        if upper[i] is not None and closes[i] > upper[i]:
            pos[i] = +1
        # enter short on close below lower band
        elif lower[i] is not None and closes[i] < lower[i]:
            pos[i] = -1
        else:
            pos[i] = 0
    elif pos[i-1] == +1:
        # exit long on close below lower band
        pos[i] = 0 if (lower[i] is not None and closes[i] < lower[i]) else +1
    else:  # pos[i-1] == -1
        # exit short on close above upper band
        pos[i] = 0 if (upper[i] is not None and closes[i] > upper[i]) else -1

# ─── 5) Slice to plotting window ───────────────────────────────────────────
xs      = dates[START_IDX:END_IDX]
ys      = closes[START_IDX:END_IDX]
upper_s = upper[START_IDX:END_IDX]
lower_s = lower[START_IDX:END_IDX]
pos_s   = pos[START_IDX:END_IDX]

# ─── 6) Plot ───────────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=FIGSIZE)

# Plot price and channels
ax.plot(xs, ys,      label="Close Price", color=COLOR_PRICE)
ax.plot(xs, upper_s, label=f"Donchian Upper ({DC_PERIOD})", color=COLOR_DC_UPPER)
ax.plot(xs, lower_s, label=f"Donchian Lower ({DC_PERIOD})", color=COLOR_DC_LOWER)

# Shade long/short regions
for i in range(len(xs)-1):
    if pos_s[i] == +1:
        ax.axvspan(xs[i], xs[i+1], facecolor=COLOR_LONG_BG, alpha=ALPHA_BG)
    elif pos_s[i] == -1:
        ax.axvspan(xs[i], xs[i+1], facecolor=COLOR_SHORT_BG, alpha=ALPHA_BG)

ax.set_title(f"Instrument {INSTRUMENT_ID}: Donchian Breakout Strategy [{START_IDX}:{END_IDX}]")
ax.set_xlabel("Time (index)")
ax.set_ylabel("Price")
ax.legend(loc="upper left")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# ─── Config ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 1      # pick instrument 0 … 49

# Index‐range to test (inclusive start, exclusive end)
START_IDX     = 500     
END_IDX       = 750     

# Donchian channel period:
DC_PERIOD     = 20

# Position sizing
INITIAL_CAPITAL = 100_000  # $
RISK_PER_TRADE  = 0.01     # 1% per trade

# ─── 1) Load closes ─────────────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v != ""]
        closes.append(vals[INSTRUMENT_ID] if len(vals)>INSTRUMENT_ID else vals[-1])
closes = np.array(closes)
dates  = np.arange(len(closes))

# ─── 2) Build synthetic OHLC for Donchian ─────────────────────────────────
opens = np.roll(closes, 1)
opens[0] = closes[0]
highs = np.maximum(opens, closes)
lows  = np.minimum(opens, closes)

# ─── 3) Donchian bands ─────────────────────────────────────────────────────
upper = np.full_like(closes, np.nan)
lower = np.full_like(closes, np.nan)
for i in range(DC_PERIOD-1, len(closes)):
    upper[i] = np.max(highs[i-DC_PERIOD+1 : i+1])
    lower[i] = np.min(lows[i-DC_PERIOD+1 : i+1])

# ─── 4) Generate positions (same logic as before) ─────────────────────────
pos = np.zeros_like(closes, dtype=int)
for i in range(1, len(closes)):
    if pos[i-1] == 0:
        if closes[i] > upper[i]:
            pos[i] = +1      # long
        elif closes[i] < lower[i]:
            pos[i] = -1      # short
    elif pos[i-1] == +1:
        pos[i] = 0 if (closes[i] < lower[i]) else +1
    elif pos[i-1] == -1:
        pos[i] = 0 if (closes[i] > upper[i]) else -1

# ─── 5) Compute daily P&L & equity curve ─────────────────────────────────
# returns from bar to bar:
rets = np.concatenate([[0], np.diff(closes) / closes[:-1]])

# Simple position‐weighted P&L:
strat_rets = pos[:-1] * rets[1:]   # we apply position from previous bar
strat_rets = np.concatenate([[0], strat_rets])

# Equity curve:
capital = INITIAL_CAPITAL
equity = [capital]
for r in strat_rets[START_IDX:END_IDX]:
    capital *= (1 + r * RISK_PER_TRADE)  # risking a fixed fraction per bar
    equity.append(capital)
equity = np.array(equity)

# ─── 6) Plot price+bands, positions, and equity curve ──────────────────────
fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(14,10), sharex=True)

# Price and Donchian bands + long/short shading
xs = dates[START_IDX:END_IDX]
ax1.plot(xs, closes[START_IDX:END_IDX], label="Close", color='tab:blue')
ax1.plot(xs, upper[START_IDX:END_IDX], label="Donchian Upper", color='tab:orange')
ax1.plot(xs, lower[START_IDX:END_IDX], label="Donchian Lower", color='tab:orange')
for i in range(len(xs)-1):
    if pos[START_IDX+i] == +1:
        ax1.axvspan(xs[i], xs[i+1], color='tab:green', alpha=0.2)
    elif pos[START_IDX+i] == -1:
        ax1.axvspan(xs[i], xs[i+1], color='tab:red', alpha=0.2)
ax1.set_ylabel("Price")
ax1.legend(loc="upper left")
ax1.set_title("Donchian Breakout & Positions")

# Position timeline
ax2.step(xs, pos[START_IDX:END_IDX], where='post', label="Position")
ax2.set_ylabel("Position")
ax2.set_ylim(-1.5,1.5)
ax2.legend(loc="upper left")
ax2.set_title("Long/Flat/Short")

# Equity curve
ax3.plot(np.arange(START_IDX, END_IDX+1), equity, color='tab:purple')
ax3.set_ylabel("Equity")
ax3.set_xlabel("Index")
ax3.set_title("Equity Curve (1% Risk per Bar)")
ax3.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# ─── CONFIG ────────────────────────────────────────────────────────────────
PRICE_FILE    = "prices.txt"
INSTRUMENT_ID = 1      # pick instrument 0…49

START_IDX     = 500     # inclusive
END_IDX       = 750     # exclusive

DC_PERIOD     = 2     # Donchian look-back
MA_PERIOD     = 15      # Simple MA look-back
ADX_PERIOD    = 24      # Wilder’s ADX
ADX_THRESH    = 0      # require ADX above this to trade

FIGSIZE       = (14, 8)
COLOR_PRICE   = 'tab:blue'
COLOR_DC      = 'tab:orange'
COLOR_MA      = 'tab:purple'
COLOR_LONG_BG = 'tab:green'
COLOR_SHORT_BG= 'tab:red'
ALPHA_BG      = 0.15

# ─── 1) LOAD CLOSES ───────────────────────────────────────────────────────
closes = []
with open(PRICE_FILE, newline='') as f:
    reader = csv.reader(f, delimiter=' ', skipinitialspace=True)
    for row in reader:
        vals = [float(v) for v in row if v!=""]
        closes.append(vals[INSTRUMENT_ID] if len(vals)>INSTRUMENT_ID else vals[-1])
closes = np.array(closes)
T      = len(closes)
dates  = np.arange(T)

# ─── 2) SYNTHETIC OHLC ─────────────────────────────────────────────────────
opens = np.roll(closes, 1); opens[0] = closes[0]
highs = np.maximum(opens, closes)
lows  = np.minimum(opens, closes)

# ─── 3) DONCHIAN CHANNEL ───────────────────────────────────────────────────
upper = np.full(T, np.nan)
lower = np.full(T, np.nan)
for i in range(DC_PERIOD-1, T):
    upper[i] = highs[i-DC_PERIOD+1:i+1].max()
    lower[i] = lows [i-DC_PERIOD+1:i+1].min()

# ─── 4) SIMPLE MA ──────────────────────────────────────────────────────────
ma = np.full(T, np.nan)
for i in range(MA_PERIOD-1, T):
    ma[i] = closes[i-MA_PERIOD+1:i+1].mean()

# ─── 5) COMPUTE ADX ────────────────────────────────────────────────────────
# 5a) True range, +DM, –DM
tr  = np.zeros(T)
pdm = np.zeros(T)
ndm = np.zeros(T)
tr[0] = highs[0] - lows[0]
for i in range(1, T):
    tr[i] = max(
        highs[i] - lows[i],
        abs(highs[i] - closes[i-1]),
        abs(lows[i]  - closes[i-1])
    )
    up  = highs[i] - highs[i-1]
    dn  = lows[i-1]  - lows[i]
    pdm[i] = up  if (up>dn and up>0) else 0.0
    ndm[i] = dn  if (dn>up and dn>0) else 0.0

# 5b) Wilder’s smoothing
atr   = np.full(T, np.nan)
pdm_s = np.full(T, np.nan)
ndm_s = np.full(T, np.nan)
# first smoothed values at ADX_PERIOD:
atr[ADX_PERIOD]   = tr[1:ADX_PERIOD+1].mean()
pdm_s[ADX_PERIOD] = pdm[1:ADX_PERIOD+1].mean()
ndm_s[ADX_PERIOD] = ndm[1:ADX_PERIOD+1].mean()
for i in range(ADX_PERIOD+1, T):
    atr[i]   = (atr[i-1]*(ADX_PERIOD-1)   + tr[i])   / ADX_PERIOD
    pdm_s[i] = (pdm_s[i-1]*(ADX_PERIOD-1) + pdm[i]) / ADX_PERIOD
    ndm_s[i] = (ndm_s[i-1]*(ADX_PERIOD-1) + ndm[i]) / ADX_PERIOD

# 5c) +DI, –DI, DX
plus_di  = 100 * pdm_s / atr
minus_di = 100 * ndm_s / atr
dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)

# 5d) ADX
adx = np.full(T, np.nan)
start_adx = 2*ADX_PERIOD
adx[start_adx] = dx[ADX_PERIOD+1:start_adx+1].mean()
for i in range(start_adx+1, T):
    adx[i] = (adx[i-1]*(ADX_PERIOD-1) + dx[i]) / ADX_PERIOD

# ─── 6) GENERATE FILTERED BREAKOUT SIGNALS ───────────────────────────────
pos = np.zeros(T, int)  # -1, 0 or +1
for i in range(1, T):
    can_long  = (closes[i]>upper[i]
                 and closes[i]>ma[i]
                 and adx[i]>ADX_THRESH)
    can_short = (closes[i]<lower[i]
                 and closes[i]<ma[i]
                 and adx[i]>ADX_THRESH)

    if pos[i-1]==0:
        pos[i] = +1 if can_long else (-1 if can_short else 0)
    elif pos[i-1]==+1:
        # exit long if channel breaks down, MA flips, or ADX drops
        pos[i] = 0 if (closes[i]<lower[i] or closes[i]<ma[i] or adx[i]<ADX_THRESH) else +1
    else:
        pos[i] = 0 if (closes[i]>upper[i] or closes[i]>ma[i] or adx[i]<ADX_THRESH) else -1

# ─── 7) SLICE FOR PLOTTING ─────────────────────────────────────────────────
xs      = dates[START_IDX:END_IDX]
cl_s    = closes[START_IDX:END_IDX]
up_s    = upper[START_IDX:END_IDX]
lo_s    = lower[START_IDX:END_IDX]
ma_s    = ma[START_IDX:END_IDX]
pos_s   = pos[START_IDX:END_IDX]

# ─── 8) PLOT ───────────────────────────────────────────────────────────────
fig, (ax1, ax2) = plt.subplots(2,1, figsize=FIGSIZE, sharex=True)

# Top: price + Donchian + MA
ax1.plot(xs, cl_s, color=COLOR_PRICE, label="Close")
ax1.plot(xs, up_s, color=COLOR_DC,    label=f"Donchian Upper({DC_PERIOD})")
ax1.plot(xs, lo_s, color=COLOR_DC,    label=f"Donchian Lower({DC_PERIOD})")
ax1.plot(xs, ma_s, color=COLOR_MA,    label=f"MA({MA_PERIOD})")
ax1.set_ylabel("Price")
ax1.set_title(f"Instr {INSTRUMENT_ID}: Filtered Donchian Breakouts [{START_IDX}:{END_IDX}]")
ax1.legend(loc="upper left")
ax1.grid(alpha=0.3)

# Bottom: price + long/short shading
ax2.plot(xs, cl_s, color=COLOR_PRICE, label="Close")
for i in range(len(xs)-1):
    if pos_s[i]==+1:
        ax2.axvspan(xs[i], xs[i+1], facecolor=COLOR_LONG_BG, alpha=ALPHA_BG)
    elif pos_s[i]==-1:
        ax2.axvspan(xs[i], xs[i+1], facecolor=COLOR_SHORT_BG,alpha=ALPHA_BG)
ax2.set_ylabel("Price")
ax2.set_xlabel("Index")
ax2.set_title("Filtered Long/Short Zones")
ax2.legend(loc="upper left")
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
import csv
import numpy as np
import matplotlib.pyplot as plt

# ─── CONFIG ────────────────────────────────────────────────────────────────
PRICE_FILE = "prices.txt"
N_PIPS     = 20       # number of perceptually important points to extract

# ─── 1) Load close‐only price matrix ─────────────────────────────────────────
prices = np.loadtxt(PRICE_FILE)  # shape: [T, N]
T, N    = prices.shape

# ─── 2) PIP extraction function ─────────────────────────────────────────────
def extract_pips(series, n_pips):
    """
    Extract 'n_pips' perceptually important points (PIPs) from a 1D series.
    """
    # Start with endpoints
    pips = [0, len(series) - 1]
    for _ in range(n_pips - 2):
        max_dist = -1
        index_to_add = None
        
        # Ensure pips sorted
        pips_sorted = sorted(pips)
        
        # Check every point not yet in pips
        for idx in range(1, len(series) - 1):
            if idx in pips:
                continue
            
            # Find segment (j, k) in pips_sorted that wraps idx
            for j, k in zip(pips_sorted[:-1], pips_sorted[1:]):
                if j < idx < k:
                    # Compute perpendicular distance from point to segment
                    x0, y0 = idx, series[idx]
                    x1, y1 = j, series[j]
                    x2, y2 = k, series[k]
                    # Line formula distance
                    num = abs((y2 - y1)*x0 - (x2 - x1)*y0 + x2*y1 - y2*x1)
                    den = np.hypot(y2 - y1, x2 - x1)
                    dist = num / den if den != 0 else 0
                    if dist > max_dist:
                        max_dist = dist
                        index_to_add = idx
                    break
        
        if index_to_add is not None:
            pips.append(index_to_add)
        else:
            break
    
    return sorted(pips)

# ─── 3) Plot each instrument with its PIPs ──────────────────────────────────
for inst in range(N):
    series = prices[:, inst]
    pips = extract_pips(series, N_PIPS)
    
    plt.figure(figsize=(8, 3))
    plt.plot(series, label=f"Instrument {inst}", color="tab:blue", linewidth=1)
    plt.scatter(pips, series[pips], color="tab:red", s=40, label="PIPs")
    plt.title(f"Instrument {inst}: Perceptually Important Points")
    plt.xlabel("Time index")
    plt.ylabel("Price")
    plt.legend(loc="upper right")
    plt.tight_layout()
    plt.show()
